<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 3:** LangChain</font>

<br>

在之前的 notebook 中，我们介绍了一些用于 LLM 应用的服务，包括外部 LLM 平台和本地托管的前端服务。这两个组件都用到了 LangChain，但目前我们还没细致的讨论它。如果您有使用 LangChain 和 LLM 的经验就再好不过了，但没有也没关系，本 notebook 会带您了解这些，以便顺利完成本课程！

本 notebook 旨在带您了解 LangChain（一个领先的大语言模型编排库）的应用，还包括之前提到的 AI Foundation Endpoints。无论您是经验丰富的开发者还是 LLM 新手，本课程都将提升您构建复杂 LLM 应用的技能。

<br>

### **学习目标：**

* 学习如何利用链（chain）和运行时（runnable）编排有趣的 LLM 系统。
* 熟悉使用 LLM 进行外部对话和内部推理。
* 能够在 notebook 中启动和运行简单的 Gradio 界面。

<br>

### **思考问题：**

* 需要哪些工具来保持信息在工作流中的传输（**下一个 notebook 的前提知识**）。
* 当您看到 `gradio` 界面时，想想您之前是否在哪里看到过这种风格的界面。有可能是 [HuggingFace Spaces](https://huggingface.co/spaces)。
* 在本节最后，您将可以将链以路由的方式传递并通过端口访问。如果您希望其它微服务能接收链的输出，应该做出哪些要求？

<br>

### **Notebook 版权声明：**

* 本 notebook 是 [**NVIDIA 深度学习培训中心**](https://www.nvidia.cn/training/)的课程[**《构建大语言模型 RAG 智能体》**](https://www.nvidia.cn/training/instructor-led-workshops/building-rag-agents-with-llms/)中的一部分，未经 NVIDIA 授权不得分发。

<br> 


### **环境设置：**

In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

<br>

### **考虑模型**

回到 [**NGC Catalog**](https://catalog.ngc.nvidia.com/ai-foundation-models)，我们能找到一些可以从环境直接调用的有趣的模型。放这些模型在这是因为它们都在实际的生产流程中被用到过，您可以看看哪些适合您的应用场景。

**课程提供的代码包括一些已经列出的模型，但如果您发现有更好的选择或者模型不再可用，您完全可以替换为其它模型。*这适用于整个课程，请记住这一点！***

----

<br>

## **第 1 部分：** LangChain 是什么？

LangChain 是一个流行的 LLM 编排库，可帮助组织一个有单个或多个 LLM 组件的系统。这个库当下非常受欢迎，并且会根据该领域的发展迅速做出变化，这意味着开发者会对 LangChain 的某些部分有丰富的经验，同时又对其它部分几乎不了解（一方面是因为有太多不同的功能，另一方面，该领域在不断迭代更新，有些功能是最近才实现的）。

此 notebook 将使用 **LangChain Expression Language (LCEL)**，从基本的链规范了解到更高级的对话管理实践，希望您能享受这趟旅程，即使是经验丰富的 LangChain 开发者也能有所收获！

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/langchain-diagram.png" width=400px/>

----

<br>

## **第 2 部分：** 链和运行时

在探索一个新库的时候，首先要关注库的核心系统是什么，以及它是怎么使用的。

在 LangChain 中，主要的构建块*曾经是*经典的**链（Chain）**：是一个执行特定操作的小型功能模块，可以跟其它链连接以构建系统。因此它可以被抽象为一个“构建块系统”，其中每个构建块都很容易创建，它们有一致的方法（`invoke`，`generate`，`stream`，等），并且可以连接成一整个系统协同工作。一些传统的链包括 `LLMChain`，`ConversationChain`，`TransformationChain`，`SequentialChain` 等等。

最近，出现了一种更易于使用且极其紧凑的规范，即 **LangChain Expression Language (LCEL)**。这种新范式依赖于另一种基础组建，即**运行时（Runnable）**，它就是一个封装函数的对象。允许将字典隐式转换为运行时，并可以通过 **pipe |** 操作符来创建一个从左到右传递数据 的运行时（比如 `fn1 | fn2` 就是一个运行时），通过这样一种简单的方式就可以创建复杂的逻辑！

下面是几个很有代表性的运行时，基于 `RunnableLambda` 类创建的：

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial

################################################################################
## Very simple "take input and return it"
identity = RunnableLambda(lambda x: x)  ## Or RunnablePassthrough works

################################################################################
## Given an arbitrary function, you can make a runnable with it
def print_and_return(x, preface=""):
    print(f"{preface}{x}")
    return x

rprint0 = RunnableLambda(print_and_return)

################################################################################
## You can also pre-fill some of values using functools.partial
rprint1 = RunnableLambda(partial(print_and_return, preface="1: "))

################################################################################
## And you can use the same idea to make your own custom Runnable generator
def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

################################################################################
## Chaining two runnables
chain1 = identity | rprint0
chain1.invoke("Hello World!")
print()

################################################################################
## Chaining that one in as well
output = (
    chain1           ## Prints "Hello World" & passes "Welcome Home!" onward
    | rprint1        ## Prints "1: Hello World" & passes "Welcome Home!" onward
    | RPrint("2: ")  ## Prints "2: Hello World" & passes "Welcome Home!" onward
).invoke("Welcome Home!")

## Final Output Is Preserved As "Welcome Home!"
print("\nOutput:", output)

----

<br>

## **第 3 部分：** 使用聊天模型的字典工作流

您可以借助运行时做很多事，但先来规范一些最佳实践是很重要的。出于几个重要原因，最简单的做法是将*字典*作为默认的变量容器。

**传递字典有助于我们按名称跟踪变量。**

由于字典允许我们传播命名变量（由键索引出的值），因此很适合用它们来锁定链组件的输出。

**LangChain 提示词需要以字典的形式提供值。**

在 LCEL 中让 LLM 链接收字典并生成字符串是非常自然的，反过来也同样轻松。是有意这样设计的，部分原因就是我们刚刚提到的那些。

<br>  

### **示例 1：** 一个简单的 LLM 链

经典 LangChain 最基本的组件之一就是接收一个**提示词**和一个 **LLM** 的 `LLMChain`：

* 提示词通常是从像 `PromptTemplate.from_template("string with {key1} and {key2}")` 这样的用于创建字符串的模板构造出来的。可以传入 `{"key1" : 1, "key2" : 2}` 这种字典，这样就能得到字符串 `"string with 1 and 2"`。
	+ 对于 `ChatNVIDIA` 聊天模型，需要使用 `ChatPromptTemplate.from_messages` 方法。
* LLM 接收字符串作为输入并返回一个生成的字符串。
	+ `ChatNVIDIA` 是用消息处理的，但也一个道理！最后用 **StrOutputParser** 就可以从消息中提取响应内容了。

下面就是上述简单聊天链的一个轻量级示例。它只做一件事，接收一个用来构造系统消息（用于指定总体目标）的字典，以及一条用户输入，返回响应。

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Simple Chat Pipeline
chat_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])

rhyme_chain = prompt | chat_llm | StrOutputParser()

print(rhyme_chain.invoke({"input" : "Tell me about birds!"}))

Output:

    Birds are quite a delightful find,
    With feathers and wings, they soar and entwine.
    In trees, they alight, with tails so bright,
    And sing their songs, with morning light.
    
    Some have beaks that curve, some have beaks that straight,
    Their chirps and chatter, fill the air and create
    A symphony sweet, of melodic sound,
    As birds take flight, their magic's all around.
    
    From robins to sparrows, to hawks on high,
    Each species unique, yet all touch the sky.
    With colors bright, and forms so grand,
    Birds are a wonder, in this world so bland.

除了按原样用代码的方式调用之外，我们还可以尝试使用 [**Gradio 界面**](https://www.gradio.app/guides/creating-a-chatbot-fast)。Gradio 是一款热门的工具，可以方便的创建自定义生成式 AI 界面！下面就展示了如何用这个示例链创建简单的 Gradio 聊天界面：

In [ ]:
import gradio as gr

#######################################################
## Non-streaming Interface like that shown above

# def rhyme_chat(message, history):
#     return rhyme_chain.invoke({"input" : message})

# gr.ChatInterface(rhyme_chat).launch()

#######################################################
## Streaming Interface

def rhyme_chat_stream(message, history):
    ## This is a generator function, where each call will yield the next entry
    buffer = ""
    for token in rhyme_chain.stream({"input" : message}):
        buffer += token
        yield buffer

## Uncomment when you're ready to try this. IF USING COLAB: Share=False is faster
gr.ChatInterface(rhyme_chat_stream).queue().launch(server_name="0.0.0.0", share=True, debug=True) 

## IMPORTANT!! When you're done, please click the Square button (twice to be safe) to stop the session.

<br>

### **示例 2：内部响应**

有时，您还希望在响应实际发送给用户之前，在背后进行一些快速推理。执行此任务时，您需要一个内置的强指令遵循先验模型。

下面是一个对句子进行分类的“零样本分类”（zero-shot classification）流程的示例。

**零样本分类链的各步骤如下：**
* 接收含有 `input` 和 `options` 两个必要键的字典。
* 传给零样本提示词，得到 LLM 的输入。
* 将该字符串传给模型来获取结果。

**任务：**选几个您认为适合该任务的模型，看看效果怎么样！具体来说：
* **试试在多个示例中表现稳定的模型。**如果格式都很容易解析且有很高的可预测性，那么这个模型应该就对了。
* **尝试寻找最快的模型！**这很重要，因为内部推理通常在外部响应返回之前一直在背后进行着。所以这是一个阻塞过程，会拖慢“面向用户”的结果生成。

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## TODO: Try out some more models and see if there are better options
instruct_llm = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")

sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

## One-shot classification prompt with heavy format assumptions.
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_msg),
    ("user", "[[The sea is awesome]]"),
    ("assistant", "boat"),
    ("user", "[[{input}]]"),
])

## Roughly equivalent as above for <s>[INST]instruction[/INST]response</s> format
zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

zsc_chain = zsc_prompt | instruct_llm | StrOutputParser()

def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

<br>  

### **示例 3：多组件链**

前面我们展示了如何通过将字典传递给 `prompt -> LLM` 链来转成一个字符串，这样的简单结构很适合用容器来构建。但是将字符串转回字典是否也同样简单？

**是的！**最简单的方法就是用 LCEL 的 *“implicit runnable”* 语法，它允许您将以字典形式组织的多个函数（包括链）作为运行时，执行时会运行每个函数并将值映射到输出字典的键。

下面的练习就用到了这个功能，同时还提供了一些实用的额外工具。

In [ ]:
################################################################################
## Example of dictionary enforcement methods
def make_dictionary(v, key):
    if isinstance(v, dict):
        return v
    return {key : v}

def RInput(key='input'):
    '''Coercing method to mold a value (i.e. string) to in-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

def ROutput(key='output'):
    '''Coercing method to mold a value (i.e. string) to out-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

################################################################################
## Common LCEL utility for pulling values from dictionaries
from operator import itemgetter

up_and_down = (
    RPrint("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | RPrint("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | RPrint("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | RPrint("D: ")
    | itemgetter("word1")
    | RPrint("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | RPrint("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)

up_and_down.invoke({"input" : "Hello World"})

In [ ]:
## NOTE how the dictionary enforcement methods make it easy to make the following syntax equivalent
up_and_down.invoke("Hello World")

----

<br>

## **第 4 部分：[练习]** Rhyme Re-themer 聊天机器人

下面是一个诗歌生成示例，展示了如何以单个智能体的形式组织两个不同的任务。这个系统跟前面简单的 Gradio 例子类似，但会在背后扩展一些样板（boiler-plate）响应和逻辑。

它的主要功能包括：
* 在第一个响应中，它会根据您的输入生成一首诗。
* 在后续的回复中，它会在保留原始诗的格式和结构的同时，修改诗的主题。

**问题：**目前，系统应该可以正常完成第一个功能，但第二个功能还没实现。

**目标：**实现 `rhyme_chat2_stream`，让智能体能正确完成这两个功能。

为了让 gradio 组件更易于使用，我们提供了一个更简洁的 `queue_fake_streaming_gradio` 方法，用标准 Python `input` 方法模拟 gradio 聊天事件循环。

In [ ]:
ChatNVIDIA.get_available_models(filter="mistralai/", list_none=False)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from copy import deepcopy

instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")  ## Feel free to change the models

prompt1 = ChatPromptTemplate.from_messages([("user", (
    "INSTRUCTION: Only respond in rhymes"
    "\n\nPROMPT: {input}"
))])

prompt2 =  ChatPromptTemplate.from_messages([("user", (
    "INSTRUCTION: Only responding in rhyme, change the topic of the input poem to be about {topic}!"
    " Make it happy! Try to keep the same sentence structure, but make sure it's easy to recite!"
    " Try not to rhyme a word with itself."
    "\n\nOriginal Poem: {input}"
    "\n\nNew Topic: {topic}"
))])

## These are the main chains, constructed here as modules of functionality.
chain1 = prompt1 | instruct_llm | StrOutputParser()  ## only expects input
chain2 = prompt2 | instruct_llm | StrOutputParser()  ## expects both input and topic

################################################################################
## SUMMARY OF TASK: chain1 currently gets invoked for the first input.
##  Please invoke chain2 for subsequent invocations.

def rhyme_chat2_stream(message, history, return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
        if entry[0] and entry[1]:
            ## If a generation occurred as a direct result of a user input,
            ##  keep that response (the first poem generated) and break out
            first_poem = entry[1]
            break

    if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

        ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage

    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        buffer = f"Sure! Here you go!\n\n"
        yield buffer

        return  ## <- TODO: Early termination for generators. Comment this out

        ########################################################################
        ## TODO: Invoke the second chain to generate the new rhymes.

        ## iterate over stream generator for second generation

        ## END TODO
        ########################################################################

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage

################################################################################
## Below: This is a small-scale simulation of the gradio routine.

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=3):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
history = [[None, "Let me help you make a poem! What would you like for me to write?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = rhyme_chat2_stream,
    history = history
)

In [ ]:
## Simple way to initialize history for the ChatInterface
chatbot = gr.Chatbot(value = [[None, "Let me help you make a poem! What would you like for me to write?"]])

## IF USING COLAB: Share=False is faster
gr.ChatInterface(rhyme_chat2_stream, chatbot=chatbot).queue().launch(debug=True, share=True)

----

<br>

## **第 5 部分：[练习]** 更深入的使用 LangChain 集成

本练习让您有机会探究一些 [**LangServer**](https://www.langchain.com/langserve) 的示例代码。具体来说是 [`frontend`](frontend) 目录以及 [`09_langserve.ipynb`](09_langserve.ipynb) notebook。

本练习需要使用课程环境。此服务仅在您提交最终项目时才需要！

* 访问 [`09_langserve.ipynb`](09_langserve.ipynb) 并运行脚本以启动具有多个可用路由的服务。
* 完成后，请验证以下 [**LangServer `RemoteRunnable`**](https://python.langchain.com/docs/langserve) 能正常工作。[`RemoteRunnable`](https://python.langchain.com/docs/langserve) 的目的是为了能轻松地将 LangChain 链托管为 API 入口，以下操作只是测试它是否可以正常工作。
	+ 如果第一次不成功，可能是操作顺序出了问题。您可以尝试重启 langserve notebook。LangServe 仍处于早期（v0.0.35），所以可能就是会遇到一些问题。
* 假设您的本地实例正常，在浏览器复制一个当前的选项卡，将地址中“/lab”开始的内容换成 `:8090`（即 `http://<...>.courses.nvidia.com:8090`）。这里包括了部署好的可供交互的 [`frontend`](frontend) 文件夹。**或者，您也可以运行下面的单元来生成链接：**

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

-----

**注意：**在这种环境中部署 LangServe API 是一种不太常规的操作，仅为给学员展示一些有趣的代码。实践中可以使用优化过的更稳定的单功能容器，以下网址包含更多相关内容：
* [**NVIDIA “支持检索增强生成的 AI 聊天机器人” 技术简介**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief)
* [**NVIDIA/GenerativeAIExamples GitHub Repo**](https://github.com/NVIDIA/GenerativeAIExamples/tree/main/RetrievalAugmentedGeneration)

-----

<br>

## **第 6 部分：** 总结

此 notebook 的目标是向您介绍 LangChain Expression Language 的模式，并让您接触一下用来提供 LLM 功能的 `gradio` 和 `LangServe` 接口！之后的 notebook 会继续本 notebook 中对 LLM 智能体开发新范式的探索。

### <font color="#76b900">**非常好！**</font>

### 后续步骤：
1. **[可选]** 花几分钟时间从 `frontend` 目录中看看部署的具体方式和底层功能。
2. **[可选]** 回顾 notebook 顶部的“思考问题”。